## Direct location with Sentinel-3 SLSTR sensor 

This notebook shows examples on direct location performed using SLSTR sensor and using DEM GETAS zarr format.

In [ ]:
import os
import json

import numpy as np

from asgard.core.frame import FrameId
from asgard.models.body import EarthBody
from asgard.models.time import TimeReference
from asgard.sensors.sentinel3 import S3SLSTRGeometry

from asgard_legacy_drivers.drivers.geojson_util import to_points
from asgard_legacy_drivers.drivers.sentinel_3_legacy import S3LegacyDriver, ExplorerDriver

### Setup variables and configure S3SLSTRGeometry

In [ ]:
# these directories are required and must be set beforehand
TEST_DIR = os.environ.get("TEST_DIR", "../tests")
SLSTR_DIR = os.path.join(TEST_DIR, "resources/S3/SLSTR")
ASGARD_DATA = os.environ.get("ASGARD_DATA", "/data/asgard")
GETAS_PATH = os.path.join(ASGARD_DATA, "ADFstatic", "S0__ADF_GETAS_20000101T000000_21000101T000000_20240529T142617.zarr")

In [ ]:
# extracting FRO orbit from 2022-10-30
orbit_file = os.path.join(
        TEST_DIR,
        "resources/S3/FRO",
        "S3A_OPER_MPL_ORBRES_20221030T000000_20221109T000000_0001.EOF",)

fro_20221030 = S3LegacyDriver.read_orbit_file(orbit_file)

In [ ]:
# extracting thermoelastic model for SLSTR
thermoelastic = S3LegacyDriver.s3_thermoelastic_tables(os.path.join(SLSTR_DIR, "GEC/SL_1_GEC_AX.nc"))

In [ ]:
# extracting geometric model for SLSTR
geom_model = S3LegacyDriver.slstr_geometry_model(os.path.join(SLSTR_DIR, "GEO/SL_1_GEO_AX.nc"))

In [ ]:
# generating image coordinates for SLSTR nadir and oblique 
coord_nad = np.zeros((10, 10, 2), dtype="int32")
for row in range(10):
    for col in range(10):
        coord_nad[row, col, 0] = 110 * col
        coord_nad[row, col, 1] = 70 * row

coord_obl = np.zeros((10, 10, 2), dtype="int32")
for row in range(10):
    for col in range(10):
        coord_obl[row, col, 0] = 110 * col
        coord_obl[row, col, 1] = 70 * row

In [ ]:
# setting S3SLSTRProduct
# creating all needed config variables 
navatt_orbit = S3LegacyDriver.read_orbit_file(
        os.path.join(SLSTR_DIR, "navatt/sample_orbit.xml")
    )

if navatt_orbit["frame"] != "EME2000":
        iers_data = S3LegacyDriver.read_iers_file(os.path.join(TEST_DIR, "resources", "207_BULLETIN_B207.txt"))
        time_model = TimeReference(iers_bulletin_b=iers_data)
        body = EarthBody(time_reference=time_model)
        navatt_orbit = body.transform_orbit(navatt_orbit, FrameId.EME2000)

navatt_attitude = S3LegacyDriver.read_attitude_file(
    os.path.join(SLSTR_DIR, "navatt/sample_attitude.xml")
)
navatt_times = np.load(os.path.join(SLSTR_DIR, "navatt/sample_timestamps_gps.npy"))
navatt_oop = np.load(os.path.join(SLSTR_DIR, "navatt/sample_oop.npy"))

slt_dir = os.path.join(SLSTR_DIR, "SLT_20221101T204936")
first_scan = 300
scan_timestamps = np.load(os.path.join(slt_dir, "sample_frame_times.npy"))
nad_first_acq = np.load(os.path.join(slt_dir, "sample_nad_first_acq.npy"))
nad_scan_angle_1km = np.load(os.path.join(slt_dir, "sample_nad_scan_angle_1km.npy"))
nad_scan_angle_05km = np.load(os.path.join(slt_dir, "sample_nad_scan_angle_05km.npy"))
obl_first_acq = np.load(os.path.join(slt_dir, "sample_obl_first_acq.npy"))
obl_scan_angle_1km = np.load(os.path.join(slt_dir, "sample_obl_scan_angle_1km.npy"))
obl_scan_angle_05km = np.load(os.path.join(slt_dir, "sample_obl_scan_angle_05km.npy"))

nb_pixels_nad = nad_scan_angle_1km.shape[1]
nb_pixels_obl = obl_scan_angle_1km.shape[1]


sample_time_array = scan_timestamps[first_scan:]

body = EarthBody()
fro_20221030_eme = body.transform_orbit(fro_20221030.copy(), FrameId.EME2000)

navatt_orbit["time_ref"] = "GPS"
navatt_attitude["times"]["GPS"] = navatt_orbit["times"]["GPS"]
navatt_attitude["time_ref"] = "GPS"

In [ ]:
# setting S3SLSTRProduct config dict
config = {
    "sat": "SENTINEL_3",
    "orbit_aux_info": {
      "orbit_state_vectors": [fro_20221030_eme],
    },
    "resources":
    {
        "dem_path": GETAS_PATH,
        "dem_type": "ZARR_GETAS",
    },
    "thermoelastic": thermoelastic,
    "geometry_model": geom_model,
    "sw_geocal": 2,
    "acquisition_times": {
        "NAD": {
            "scan_times": {"offsets": sample_time_array},
            "nb_pixels": nb_pixels_nad,
            "first_acquisition": nad_first_acq[first_scan:],
        },
        "OBL": {
            "scan_times": {"offsets": sample_time_array},
            "nb_pixels": nb_pixels_obl,
            "first_acquisition": obl_first_acq[first_scan:],
        },
    },
    "navatt": {
        "orbit": navatt_orbit,
        "attitude": navatt_attitude,
        "times": {"offsets": navatt_times},
        "oop": navatt_oop,
    },
    "scan_encoder": {
        "NAD": {
            "1KM": nad_scan_angle_1km[first_scan:, :],
            "05KM": nad_scan_angle_05km[first_scan:, :],
        },
        "OBL": {
            "1KM": obl_scan_angle_1km[first_scan:, :],
            "05KM": obl_scan_angle_05km[first_scan:, :],
        },
    },
}

In [ ]:
# instantiating a S3SLSTRProduct
slstr = S3SLSTRGeometry(**config)

### Direct location with GETAS zarr

In [ ]:
# running direct locations using nadir view geometry 
gnd_nad, _ = slstr.direct_loc(coord_nad, geometric_unit="NAD/1KM/0")

In [ ]:
# running direct locations using oblique view geometry 
gnd_obl, _ = slstr.direct_loc(coord_obl, geometric_unit="OBL/1KM/0")

### Visualize direct location on map 

In [ ]:
## Nadir data points case :
# converting ground positions gnd_point to GEOJson MultiPoint at indicated path
# create live directory if it doesn't exist
live_dir = os.path.join(TEST_DIR, "outputs/live")
if not os.path.exists(live_dir):
    os.makedirs(live_dir)


path_nad = os.path.join(TEST_DIR, "outputs/live/slstr_nad_points.geojson")
# conversion to GeoJson foarmat
to_points(gnd_nad, path_nad)

In [ ]:
## Oblique data points case :
# converting ground positions gnd_point to GEOJson MultiPoint at indicated path
path_obl = os.path.join(TEST_DIR, "outputs/live/slstr_obl_points.geojson")
# conversion to GeoJson foarmat
to_points(gnd_obl, path_obl)

In [ ]:
# opening file and extracting coordinates for Nadir case
with open(path_nad, 'r') as f:
    json_nad = json.load(f)
coordinates_nad = json_nad['features'][0]['geometry']['coordinates']

# opening file and extracting coordinates for Oblique case
with open(path_obl, 'r') as f:
    json_obl = json.load(f)
coordinates_obl = json_obl['features'][0]['geometry']['coordinates']

In [ ]:
import folium

# initializing the locations map for Nadir case
m = folium.Map(location=coordinates_nad[0], zoom_start=4)
# iterate through the nadir coordinates and add them as circle-shaped markers to the map
for feature in coordinates_nad:
    coordinates = feature
    marker = folium.CircleMarker(location=coordinates, radius=5, color='blue', fill=True, fill_color='blue')
    marker.add_to(m)

# iterate through the oblique coordinates and add them as circle-shaped markers to the map
for feature in coordinates_obl:
    coordinates = feature
    marker = folium.CircleMarker(location=coordinates, radius=5, color='green', fill=True, fill_color='blue')
    marker.add_to(m)


# Create kw dict config in order to create rectangle to represent DEM tiles
kw = {
    "color": "black",
    "line_cap": "round",
    "fill": True,
    "fill_color": "blue",
    "weight": 5,
    "popup" : "tile90m_1"
}

# bounds1 = np.rad2deg(np.array([[0.407243, 0.349065850], [0.552687596, 0.49450995]]))
# bounds2 = np.rad2deg(np.array([[0.407243, 0.058177641], [0.552687596, 0.2036217]]))
bounds1 = np.rad2deg(np.array([[0.407243, 0.2036217], [0.552687596, 0.349065850]]))
bounds2 = np.rad2deg(np.array([[0.261799387799, 0.058177641], [0.407243, 0.2036217]]))
bounds3 = np.rad2deg(np.array([[0.261799387799, 0.2036217], [0.407243, 0.349065850]]))
bounds4 = np.rad2deg(np.array([[0.261799387799, 0.349065850], [0.407243, 0.49450995]]))
bounds5 = np.rad2deg(np.array([[0.11635528, 0.058177641], [0.261799387799, 0.2036217]]))
bounds6 = np.rad2deg(np.array([[0.11635528, 0.2036217], [0.261799387799, 0.349065850]]))
bounds7 = np.rad2deg(np.array([[0.11635528, 0.349065850], [0.261799387799, 0.49450995]]))

folium.Rectangle(
    bounds=bounds1,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)

kw["popup"] = "tile2"
folium.Rectangle(
    bounds=bounds2,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)

kw["popup"] = "tile3"
folium.Rectangle(
    bounds=bounds3,
    line_join="round",
    dash_array="1, 1",

    **kw,
).add_to(m)

kw["popup"] = "tile4"
folium.Rectangle(
    bounds=bounds4,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)

kw["popup"] = "tile5"
folium.Rectangle(
    bounds=bounds5,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)

kw["popup"] = "tile6"
folium.Rectangle(
    bounds=bounds6,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)

kw["popup"] = "tile7"
folium.Rectangle(
    bounds=bounds7,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)


# save map for Nadir case 
m.save('map_json_nad.html')

In [ ]:
# plot resulting map
m